In [2]:
# Визуализации

from matplotlib import pyplot as plt
import seaborn as sns

In [3]:
plt.style.use("bmh")
plt.rcParams["axes.titlesize"] = "large"

In [4]:
# Общеe

import os
import tqdm
import pathlib
import numpy as np
import pandas as pd

tqdm.tqdm.pandas()
pd.set_option('display.max_columns', None)

In [5]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso, Ridge
from sklearn.svm import SVR
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
import pickle
from sklearn.preprocessing import StandardScaler

# Загрузка данных

Данные хранятся в формате Parquet: читается быстрее, места занимает меньше.

In [6]:
data = pd.read_parquet("train.parquet")
data.head()

,feature0,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12,feature13,feature14,feature15,feature16,feature17,feature18,feature19,feature20,feature21,feature22,feature23,feature24,target0,target1
0,32.910908,17.376350,77.557840,2.929855,gas1,20.487150,57.633085,49.245392,-44.124381,23.400064,-28.640648,-10.995420,40.856313,29.912800,33.168114,18.769718,41.812507,14.203652,9.689942,17.951627,24.198589,102.448710,110.529868,56.817260,12.887802,27.050891,6.502743
1,41.263782,22.419445,47.945514,-25.847472,gas2,21.461239,3.474080,49.659980,-13.553188,-13.047593,-12.445940,16.657368,42.453280,99.792375,127.159321,-13.180244,50.722470,-14.451904,-0.906120,105.724742,-9.435222,-16.060150,37.286110,61.224272,39.822424,84.127890,76.578716
2,25.580283,17.376350,77.654180,2.799411,gas1,20.487150,28.938295,49.245392,-44.124381,20.610679,-28.641150,-1.886830,40.927941,16.340674,60.760647,18.658501,52.731646,14.395054,9.779019,17.951638,24.999453,101.728213,111.471534,52.664304,12.887802,22.080133,3.036043
3,33.756900,17.376350,73.049625,2.953982,gas1,20.487150,28.932311,49.245392,-44.124381,18.107963,-28.642094,16.284415,40.869406,43.707663,48.037510,18.771030,52.305692,14.184314,9.798969,17.951675,23.990300,101.312113,115.589451,56.840719,12.887802,30.234082,8.910795
4,4.223732,38.772534,48.015553,-25.843943,gas2,24.635721,12.011581,51.030938,84.244199,-17.735680,40.395582,16.557978,65.988300,91.081310,150.949700,-13.172169,51.012294,-14.439953,8.262354,90.187207,-2.901661,-16.060150,37.398779,40.488468,128.295838,71.128092,50.475082


Проверяем, что с пропусками:

In [7]:
data.isnull().any().mean()

0.0

В данных есть один категориальный признак: `feature4`. Он может принимать значения `gas1` и `gas2`. Таргета два: `target0` и `target1`.

In [8]:
data['target_class'] = (data['feature4'] == 'gas1').astype(int)
data.drop(labels=['feature4'], axis=1, inplace=True)
key_cols = ['target0', 'target1', 'target_class']
cols_for_features = ['feature' + str(i) for i in range(25) if i != 4]

Построим модель Ridge (target0 gas=2)

In [ ]:
current_target = 'target0'
current_target_class = 0
current_train = data[data['target_class'] ==
                     current_target_class].reset_index(drop=True)
model = Pipeline([('polynom', PolynomialFeatures(degree=4)),
 ('ss', StandardScaler(with_std=True)),
  ('lasso', Ridge(alpha=5))])
model.fit(current_train[cols_for_features], current_train[current_target])
pickle.dump(model,
            open('model_' + current_target + '_for_target_' +
                 str(current_target_class) + '_big.pkl', 'wb'))

Построим модель Lasso (target0 gas=1)

In [ ]:
current_target = 'target0'
current_target_class = 1
current_train = data[data['target_class'] ==
                     current_target_class].reset_index(drop=True)
model = Pipeline([('polynom', PolynomialFeatures(degree=3)),
                    ('model_ridge', Lasso(alpha=1))])
model.fit(current_train[cols_for_features], current_train[current_target])
pickle.dump(model,
            open('model_' + current_target + '_for_target_' +
                 str(current_target_class) + '.pkl', 'wb'))

SVR (target0, gas=1):

In [ ]:
%%time
current_target = 'target0'
current_target_class = 1
current_train = data[data['target_class'] ==
                     current_target_class].reset_index(drop=True)
model = Pipeline([('model_ridge', SVR(kernel='poly', degree=4, C=3.0))])
model.fit(current_train[cols_for_features], current_train[current_target])
pickle.dump(model,
            open('model_' + current_target + '_for_target_' +
                 str(current_target_class) + '_svr_d4_C3.pkl', 'wb'))

Построим модель Lasso (target1 gas=2)

In [ ]:
current_target = 'target1'
current_target_class = 0
current_train = data[data['target_class'] ==
                   current_target_class].reset_index(drop=True)
model = Pipeline([('polynom', PolynomialFeatures(degree=3)),
 ('model_ridge', Lasso(alpha=1))])
model.fit(current_train[cols_for_features], current_train[current_target])
pickle.dump(model,
            open('model_' + current_target + '_for_target_' +
                 str(current_target_class) + '.pkl', 'wb'))

Построим модель Ridge (target1 gas=2)

In [ ]:
current_target = 'target1'
current_target_class = 0
current_train = data[data['target_class'] ==
                     current_target_class].reset_index(drop=True)
model = Pipeline([('polynom', PolynomialFeatures(degree=3)),
 ('ss', StandardScaler(with_std=True)),
  ('lasso', Ridge(alpha=0.01))])
model.fit(current_train[cols_for_features], current_train[current_target])
pickle.dump(model,
            open('model_' + current_target + '_for_target_' +
                 str(current_target_class) + '_ss_ridge_001.pkl', 'wb'))

SVR (target1, gas=1):

In [ ]:
%%time
current_target = 'target1'
current_target_class = 1
current_train = data[data['target_class'] ==
                     current_target_class].reset_index(drop=True)
model = Pipeline([('model_ridge', SVR(kernel='poly', degree=5, C=3.0))])
model.fit(current_train[cols_for_features], current_train[current_target])
pickle.dump(model,
            open('model_' + current_target + '_for_target_' +
                 str(current_target_class) + '_svr_C_3.pkl', 'wb'))

Lasso squares

In [ ]:
%%time
current_target = 'target1'
current_target_class = 1
train11 = data.loc[data['target_class'] ==
                   current_target_class, :].reset_index(drop=True)
train11_target = train11[current_target].values
train11 = train11[cols_for_features]
for i in range(len(cols_for_features)):
  train11[cols_for_features[i] + '_2'] = train11[cols_for_features[i]]
model = Pipeline([('polynom', PolynomialFeatures(degree=3)),
 ('ss', StandardScaler()),
  ('lasso', Lasso(alpha=0.0001))])
model.fit(train11, train11_target)
pickle.dump(model,
            open('model_' + current_target + '_for_target_' +
              str(current_target_class) + '_lasso_squares.pkl', 'wb'))

# Создаём сабмит

In [ ]:
%%writefile predict.py
import warnings
warnings.filterwarnings('ignore')
import pathlib
import pandas as pd
import numpy as np
import pickle
from multiprocessing import Pool

DATA_DIR = pathlib.Path(".")

# Lasso
MODEL_FILE_target01 =
pathlib.Path(__file__).parent.joinpath("model_target0_for_target_1
.pkl")
MODEL_FILE_target10 =
pathlib.Path(__file__).parent.joinpath("model_target1_for_target_0
.pkl")

# Ridge
MODEL_FILE_target00_ridge =
pathlib.Path(__file__).parent.joinpath("model_target0_for_target_0
_big.pkl")
MODEL_FILE_target10_ridge =
pathlib.Path(__file__).parent.joinpath("model_target1_for_target_0
_ss_ridge_001.pkl")

# SVR
MODEL_FILE_target01_svr =
pathlib.Path(__file__).parent.joinpath("model_target0_for_target_1
_svr_d4_C3.pkl")
MODEL_FILE_target11_svr =
pathlib.Path(__file__).parent.joinpath("model_target1_for_target_1
_svr_C_3.pkl")

# Square
MODEL_FILE_target11_square =
pathlib.Path(__file__).parent.joinpath("model_target1_for_target_1
_lasso_squares.pkl")

models_dict = {}
models_dict['target0'] = {}
models_dict['target1'] = {}
models_dict['target0'][0] = {}
models_dict['target0'][0]['ridge'] = MODEL_FILE_target00_ridge
models_dict['target0'][1] = {}
models_dict['target0'][1]['lasso'] = MODEL_FILE_target01
models_dict['target0'][1]['svr'] = MODEL_FILE_target01_svr
models_dict['target1'][0] = {}
models_dict['target1'][0]['lasso'] = MODEL_FILE_target10
models_dict['target1'][0]['ridge'] = MODEL_FILE_target10_ridge
models_dict['target1'][1] = {}
models_dict['target1'][1]['svr'] = MODEL_FILE_target11_svr
models_dict['target1'][1]['square'] = MODEL_FILE_target11_square

def predict_batch(df: pd.DataFrame) -> pd.DataFrame:
  df['target_class'] = (df['feature4'] == 'gas1') * 1
  df.drop(labels=['feature4'], axis=1, inplace=True)
  key_cols = ['target0', 'target1', 'target_class']
  cols_for_features = ['feature' + str(i) for i in range(25) if i != 4]

  predictions = df[['target_class']]

  current_target = 'target0'
  current_target_class = 0
  current_test = df[df['target_class'] == current_target_class]
  model_ridge = pickle.load(open(models_dict[current_target]
    [current_target_class]['ridge'], 'rb'))
  predictions.loc[current_test.index, current_target] =
    model_ridge.predict(current_test[cols_for_features])


  current_target = 'target0'
  current_target_class = 1
  current_test = df[df['target_class'] == current_target_class]
  model_lasso = pickle.load(open(models_dict[current_target]
    [current_target_class]['lasso'], 'rb'))
  model_svr = pickle.load(open(models_dict[current_target]
    [current_target_class]['svr'], 'rb'))
  predictions.loc[current_test.index, current_target] =\
  (0.5 * model_lasso.predict(current_test[cols_for_features]) +\
  0.5 * model_svr.predict(current_test[cols_for_features]))

  current_target = 'target1'
  current_target_class = 0
  current_test = df[df['target_class'] == current_target_class]
  model_lasso = pickle.load(open(models_dict[current_target]
    [current_target_class]['lasso'], 'rb'))
  model_ridge = pickle.load(open(models_dict[current_target]
    [current_target_class]['ridge'], 'rb'))
  predictions.loc[current_test.index, current_target] =\
  (0.5 * model_lasso.predict(current_test[cols_for_features]) +\
    0.5 * model_ridge.predict(current_test[cols_for_features]))

  current_target = 'target1'
  current_target_class = 1
  current_test = df.loc[df['target_class'] == current_target_class]
  model_svr = pickle.load(open(models_dict[current_target]
    [current_target_class]['svr'], 'rb'))
  model_square = pickle.load(open(models_dict[current_target]
    [current_target_class]['square'], 'rb'))
  test11 = df.loc[df['target_class'] == current_target_class, cols_for_features]
  for i in range(len(cols_for_features)):
    test11[cols_for_features[i] + '_2'] = test11[cols_for_features[i]] *\
     test11[cols_for_features[i]]

  predictions.loc[current_test.index, current_target] =\
      (0.5 * model_square.predict(test11) +\
      0.5 * model_svr.predict(current_test[cols_for_features]))

  predictions = predictions[['target0', 'target1']]
  predictions['target0'] = np.clip(predictions['target0'], 0, 100)
  predictions['target1'] = np.clip(predictions['target1'], 0, 100)
  return predictions

def predict(df: pd.DataFrame) -> pd.DataFrame:
  pool = Pool(4)
  prediction_table = pool.map(predict_batch, [df.iloc[i * 5_000: (i + 1) *\
   5_000] for i in range(df.shape[0] // 5_000 + 1)])
  return pd.concat(prediction_table).sort_index()